In [1]:
#!/usr/bin/env python
"""A short and simple example experiment with restarts.

The code is fully functional but mainly emphasises on readability.
Hence it neither produces any progress messages (which can be very
annoying for long experiments) nor provides batch distribution,
as `example_experiment.py` does.

To apply the code to a different solver, `fmin` must be re-assigned or
re-defined accordingly. For example, using `cma.fmin` instead of
`scipy.optimize.fmin` can be done like::

    import cma
    def fmin(fun, x0):
        return cma.fmin(fun, x0, 2, {'verbose':-9})

"""
from __future__ import division, print_function
import cocoex, cocopp  # experimentation and post-processing modules
import scipy.optimize  # to define the solver to be benchmarked
from numpy.random import rand  # for randomised restarts
import os, webbrowser  # to show post-processed results in the browser

In [2]:
### input
output_folder = "Static_100"
fmin = scipy.optimize.fmin
max_nfe = 1000# increase to 10, 100, ...

# # fmin is re-defined to call our solver
# def fmin(fun, lbounds, ubounds, dim, budget):
#     result = solvers.franken10(50, fun, lbounds, ubounds, dim, 1000)
#     return result

In [4]:
### prepare
suite = cocoex.Suite("bbob", "", "function_indices:1,15 dimensions:20,40 instance_indices:1-10")
observer = cocoex.Observer("bbob", "result_folder: " + output_folder)
minimal_print = cocoex.utilities.MiniPrint()

In [9]:
def get_best_indv():
    pdlog = process_training_log()
    filter_precision = pdlog[pdlog['PRECISION'] == pdlog['PRECISION'].min()]
    
    generation = -1
    fitness = 0
    for i, row in filter_precision.filter(regex="FIT_INDV").iterrows():
        bestfitness = row.max()
    
        if bestfitness >= fitness:
            generation = i
            fitness    = bestfitness
            
    print("Best individual at generation :", generation, "(",experiment_name,")")
    return generation

In [8]:
def process_training_log():
    #load data
    data_log = pd.read_csv("./results/"+experiment_name+"/log.csv", header=None)

    #header
    header = ["GENERATIONS", "PRECISION", "FIT_MEDIAN"]
    for i in range(1, data_log.shape[1] - 2):
        header.append("FIT_INDV_"+str(i))
    data_log.columns = header
    
    return data_log

In [5]:
### process best individual
experiment_name = "T500_T10_E1"
file = open("./results/"+experiment_name+"/"+str(get_best_indv())+".txt", 'r')
best = file.readlines()
file.close()
code = "import numpy as np\nfrom src.src.solution import Solution\nimport src.src.operators as op\n"
for line in best[6:-8]:    code += line
code

'import numpy as np\nfrom src.src.solution import Solution\nimport src.src.operators as op\ndef ge(n, my_func, bounds, dimension, max_nfe):\n  Solution.setProblem(my_func, bounds, dimension, maximize=False)\n  Solution.repair = op.repair_truncate\n  X = Solution.initialize(n)\n  for Xi in X:    Xi.setX(op.init_random(*Solution.bounds, Solution.dimension))\n  [Xi.getFitness() for Xi in X]\n  Solution.updateHistory(X)\n  while Solution.nfe < max_nfe:\n    U = X\n    #Round 1\n    S1 = op.select_current(X)\n    U = op.w_pso(S1, w=0.50, c1=0.00, c2=0.75)\n    X  = U\n    #Round 2\n    S1 = op.select_tournament(U, n=1, k=int(n*0.10))\n    S2 = op.select_tournament(X, n=1, k=int(n*0.75))\n    U  = op.w_crx_blend2(S1, S2, alpha=0.50)\n    X  = op.replace_if_best(X, U)\n    #Round Drop\n    X = op.drop_worst(X, pr=0.10, k=1) \n    [Xi.getFitness() for Xi in X]\n  return X\nge(400, my_func, bounds, dimension, max_nfe)\nXXX_output_XXX = Solution.best.getFitness()\n'

In [6]:
### go
for problem in suite:  # this loop will take several minutes or longer
    problem.observe_with(observer)  # generates the data for cocopp post-processing
    x0 = problem.initial_solution
    # apply restarts while neither the problem is solved nor the budget is exhausted
    while (problem.evaluations < max_nfe
           and not problem.final_target_hit):
        d = {
            "max_nfe"  : max_nfe, 
            "dimension": problem.dimension,
            "my_func"  : problem,
            "bounds"   : (problem.lower_bounds[0], problem.upper_bounds[0])
            }
        exec(code, d)
        x0 = problem.lower_bounds + ((rand(problem.dimension) + rand(problem.dimension)) *
                    (problem.upper_bounds - problem.lower_bounds) / 2)
    minimal_print(problem, final=problem.index == len(suite) - 1)

20D: ....................
40D: ....................

In [7]:
### post-process data
cocopp.main(observer.result_folder)  # re-run folders look like "...-001" etc
webbrowser.open("file://" + os.getcwd() + "/ppdata/index.html")

Post-processing (1)
  Using:
    ./exdata/Static_100-001

Post-processing (1)
  Will generate output data in folder ppdata/Static_100-001
    this might take several minutes.
Scaling figures...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...


/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/pproc.py:1103: UserWarning:   less than 15 instances in [1, 2, 3, 4, 5, 71, 72, 73, 74, 75] (f1, 20D)
  str(self.dim) + 'D)')
/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterator):


  Data consistent according to consistency_check() in pproc.DataSet
  using: /anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/refalgs/best2009-bbob.tar.gz
  done (Tue Mar 26 17:51:31 2019).
  done (Tue Mar 26 17:51:33 2019).
Generating LaTeX tables...
  done (Tue Mar 26 17:51:33 2019).
ECDF graphs...


/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/testbedsettings.py:175: ResourceWarning: unclosed file <_io.TextIOWrapper name='/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/bbob-benchmarkinfos.txt' mode='r' encoding='UTF-8'>
  self.info_filename)).readlines():


  done (Tue Mar 26 17:51:38 2019).
ECDF graphs per function...
  done (Tue Mar 26 17:51:43 2019).
aRT loss ratio figures and tables...
  done (Tue Mar 26 17:51:45 2019).
Output data written to folder /Users/annabogdanova/Documents/Codes/SwarmGE/GECCO19/ppdata/Static_100-001
Setting changes in `cocopp.genericsettings` compared to default:
    simulated_runlength_bootstrap_sample_size: from 10 to 10.098990100989901
ALL done (Tue Mar 26 17:51:45 2019).


True